# L2: Interactive AI Applications: Building a Simple AI Role Playing Game (RPG)

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.<br>
<span style="font-size: larger;">To maintain consistency, the notebooks are run with a 'world state' consistent with the video at the start of each notebook.</span></p>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

## Create a Game UI with Gradio

In [2]:
import gradio as gr
import os
demo = None #added to allow restart

def start_game(main_loop, share=False):
    # added code to support restart
    global demo
    # If demo is already running, close it first
    if demo is not None:
        demo.close()

    demo = gr.ChatInterface(
        main_loop,
        chatbot=gr.Chatbot(height=250, placeholder="Type 'start game' to begin"),
        textbox=gr.Textbox(placeholder="What do you do next?", container=False, scale=7),
        title="AI RPG",
        # description="Ask Yes Man any question",
        theme="soft",
        examples=["Look around", "Remember what you've learn about this", "Continue the story"],
        cache_examples=False,
        retry_btn="Retry",
        undo_btn="Undo",
        clear_btn="Clear",
                           )
    demo.launch(share=share, server_name="0.0.0.0")

def test_main_loop(message, history):
    return 'Entered Action: ' + message

start_game(test_main_loop)

ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 7860): address already in use


Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


## Generating an Initial Start

In [4]:
from helper import load_world, save_world
from together import Together
from helper import get_together_api_key, load_env

client = Together(api_key=get_together_api_key())

world = load_world('../shared_data/Argentina_med_2010.json')
health_center = world['health_centers']['San Rafael Health Post']
department = health_center['departments']["Department of Biomedical Engineering and Device Development"]
character = department['npcs']['Dr. Maria Sanchez']

In [5]:
system_prompt = """You are an AI Game master. Your job is to create a 
start to an adventure based on the world, health_center, department and character 
a player is playing as. 
Instructions:
You must only use 2-4 sentences \
Write in second person. For example: "You are Jack" \
Write in present tense. For example "You stand at..." \
First describe the character and their backstory. \
Then describes where they start and what they see around them."""
world_info = f"""
World: {world}
Health Center: {health_center}
Department: {department}
Your Character: {character}
"""

In [6]:
model_output = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    temperature=1.0,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info + '\nYour Start:'}
    ],
)


In [7]:
start = model_output.choices[0].message.content
print(start)
world['start'] = start
#save_world(world, '../shared_data/Kyropeia.json')  # preserve video version
save_world(world, '../shared_data/Argentina_med_2010.json')

You are Dr. Maria Sanchez, a 35-year-old physician at San Rafael Health Post. You stand in the middle of the health post's makeshift laboratory, surrounded by shelves of dusty medical texts and makeshift workstations cluttered with DIY medical devices. The air is thick with the smell of disinfectant and the sound of generators humming in the background. You've just finished a long shift, but your mind is still racing with the challenges of treating patients in this resource-constrained setting. You think about the patient you lost last week, and the guilt that still weighs on you. You're determined to find a way to make a difference, to innovate and improve healthcare in the Pampas region. You glance around the lab, taking in the faces of your colleagues, Dr. Leonardo Marquez and Sofia Gutierrez, who are engrossed in their own projects. What do you do next?


## Creating the Main Action Loop

In [8]:
def run_action(message, history, game_state):
    
    if(message == 'start game'):
        return game_state['start']

    system_prompt = """ You are an AI Game master. Your job is to write what \
                        happens next in a player's adventure game.\
                        Instructions: \
                        You must on only write 1-3 sentences in response. \
                        Always write in second person present tense. \
                        Ex. (You look north and see...)"""
    
    world_info = f"""   World: {game_state['world']}
                        Health Center: {game_state['health_center']}
                        Department: {game_state['department']}
                        Your Character:  {game_state['character']}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info}
    ]
    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})

    messages.append({"role": "user", "content": message})
    model_output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=messages
    )
    
    result = model_output.choices[0].message.content
    return result

In [9]:
game_state = {
    "world": world['description'],
    "health_center": health_center['description'],
    "department": department['description'],
    "character": character['description'],
    "start": start,
}

def main_loop(message, history):
    return run_action(message, history, game_state)

## Launch and Share!

In [10]:
start_game(main_loop, True)

Closing server running on port: 7861


ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 7860): address already in use


Running on local URL:  http://0.0.0.0:7861
Running on public URL: https://78320fa814575f7b09.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
